In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, explained_variance_score, mean_absolute_error

In [3]:
preds = {'rxtr': 'reactor', 'burn' : 'burnup', 'cool' : 'cooling', 'enri' : 'enrichment'}
algs = ['knn', 'dtree']
fracs = ['0.1', '0.3', '1.0']
mll_frac = '1.0'
learn29 = {}
base = '~/sims_n_results/final_sims_nov2020/'
sci_path = base + 'scikit/sfco/'
path_end = '_nuc29_ext_test_compare.csv'
for frac in fracs:
    for p, pred in preds.items():
        for alg in algs:
            key = p + frac + '_' + alg
            mid = pred + '_' + alg + '_tset' + frac
            learn29[key] = pd.read_csv(sci_path + mid + path_end, index_col=0)

mll29 = pd.read_csv(base + 'mll/sfco/Job0_unc0.05/Job0_unc0.05.csv', index_col=0)

In [4]:
learn29['rxtr0.1_knn'].head(1)

,TrueY,kNN
0,pwr,bwr


In [6]:
Algs = ['kNN', 'DTree']
cols = Algs + ['MaxLL']
cols

['kNN', 'DTree', 'MaxLL']

In [9]:
rxtr29 = pd.DataFrame(columns=cols)
for frac in fracs:
    scores29 = []
    for col in cols:
        if col in Algs:
            key = 'rxtr' + frac
            if 'kNN' in col:
                key = key + '_knn'
                scores29.append(accuracy_score(learn29[key]['TrueY'], learn29[key]['kNN']))
            else:
                key = key + '_dtree'
                scores29.append(accuracy_score(learn29[key]['TrueY'], learn29[key]['DTree']))
        else:
            if frac == mll_frac:
                scores29.append(accuracy_score(mll29['ReactorType'], mll29['pred_ReactorType']))
            else:
                scores29.append(np.nan)
    rxtr29.loc[frac] = scores29
print('Reactor Type, 29 Nuclides: \n')
rxtr29

Reactor Type, 29 Nuclides: 



,kNN,DTree,MaxLL
0.1,0.582178,0.443564,NaN
0.3,0.570297,0.172277,NaN
1.0,0.584158,0.512871,0.718812


In [10]:
llparam = { 'burn' : ['Burnup', 'pred_Burnup'],
            'cool' : ['CoolingTime', 'pred_CoolingTime'],
            'enri' : ['Enrichment', 'pred_Enrichment'] }

In [15]:
def explained_variances(param):
    df29 = pd.DataFrame(columns=cols)
    for frac in fracs:
        scores29 = []
        for col in cols:
            if col in cols:
                key = param + frac
                if 'kNN' in col:
                    key = key + '_knn'
                    scores29.append(explained_variance_score(learn29[key]['TrueY'], learn29[key]['kNN']))
                else:
                    key = key + '_dtree'
                    scores29.append(explained_variance_score(learn29[key]['TrueY'], learn29[key]['DTree']))
            else:
                if frac == mll_frac:
                    scores29.append(explained_variance_score(mll29[llparam[param][0]], mll29[llparam[param][1]]))
                else:
                    scores29.append(np.nan)
        df29.loc[frac] = scores29
    print(llparam[param][0] + ', ExpVar, 29 nuclides: \n')
    return df29

In [16]:
def mean_abs_error(param):
    df29 = pd.DataFrame(columns=cols)
    for frac in fracs:
        scores29 = []
        for col in cols:
            if col in cols:
                key = param + frac
                if 'kNN' in col:
                    key = key + '_knn'
                    scores29.append(mean_absolute_error(learn29[key]['TrueY'], learn29[key]['kNN']))
                else:
                    key = key + '_dtree'
                    scores29.append(mean_absolute_error(learn29[key]['TrueY'], learn29[key]['DTree']))
            else:
                if frac == mll_frac:
                    scores29.append(mean_absolute_error(mll29[llparam[param][0]], mll29[llparam[param][1]]))
                else:
                    scores29.append(np.nan)
        df29.loc[frac] = scores29
    print(llparam[param][0] + ', MAE, 29 nuclides: \n')
    return df29

In [17]:
enri29 = explained_variances('enri')
enri29

Enrichment, ExpVar, 29 nuclides: 



,kNN,DTree,MaxLL
0.1,-1.186659,0.655351,0.655351
0.3,-1.459952,0.709187,0.709187
1.0,-1.475372,0.721632,0.721632


In [18]:
enri29 = mean_abs_error('enri')
enri29

Enrichment, MAE, 29 nuclides: 



,kNN,DTree,MaxLL
0.1,1.678349,0.348291,0.348291
0.3,1.721205,0.330467,0.330467
1.0,1.735405,0.329612,0.329612


In [19]:
burn29 = explained_variances('burn')
burn29

Burnup, ExpVar, 29 nuclides: 



,kNN,DTree,MaxLL
0.1,-0.277694,0.195675,0.195675
0.3,0.034710,0.186354,0.186354
1.0,-0.035110,0.242814,0.242814


In [20]:
burn29 = mean_abs_error('burn')
burn29

Burnup, MAE, 29 nuclides: 



,kNN,DTree,MaxLL
0.1,15152.352107,14211.989630,14211.989630
0.3,14076.621187,14283.393749,14283.393749
1.0,14259.941405,14027.794601,14027.794601
